# Tensorflow tf.data API

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
data = pd.read_csv("heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
file_data = tf.keras.utils.get_file(origin="https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?select=heart.csv")

   8192/Unknown - 0s 0s/step

In [31]:
heart_csv_ds = tf.data.experimental.make_csv_dataset("heart.csv", batch_size=12,
                                                     label_name="target",
                                                     num_epochs=1)

In [32]:
heart_csv_ds

<PrefetchDataset element_spec=(OrderedDict([('\ufeffage', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('sex', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('cp', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('trestbps', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('chol', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('fbs', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('restecg', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thalach', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('exang', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('oldpeak', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('slope', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('ca', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thal', TensorSpec(shape=(None,), dtype=tf.int32, name=None))]), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [33]:
for batch, label in heart_csv_ds.take(1):
    for key, value in batch.items():
        print(f"{key:20s}: {value}")
    print("\n", f"{'label':20s}: {label}")

﻿age                : [62 62 45 55 39 60 57 57 59 65 56 56]
sex                 : [1 0 0 0 0 1 0 1 1 0 1 1]
cp                  : [0 2 0 1 2 0 0 0 0 2 2 0]
trestbps            : [120 130 138 135  94 117 120 110 110 155 130 130]
chol                : [267 263 236 250 199 230 354 335 239 269 256 283]
fbs                 : [0 0 0 0 0 1 0 0 0 0 1 1]
restecg             : [1 1 0 0 1 1 1 1 0 1 0 0]
thalach             : [ 99  97 152 161 179 160 163 143 142 148 142 103]
exang               : [1 0 1 0 0 1 1 1 1 0 1 1]
oldpeak             : [1.8 1.2 0.2 1.4 0.  1.4 0.6 3.  1.2 0.8 0.6 1.6]
slope               : [1 1 1 1 2 2 2 1 1 2 1 0]
ca                  : [2 1 0 0 0 2 0 1 1 0 1 0]
thal                : [3 3 2 2 2 3 2 3 3 2 1 3]

 label               : [0 0 1 1 1 0 1 0 0 1 0 0]


In [34]:
%%time
for i, (batch, label) in enumerate(heart_csv_ds.repeat(20)):
    if i % 40 == 0:
        print('.', end='')
print()

.............
CPU times: total: 328 ms
Wall time: 204 ms


In [43]:
from tensorflow import feature_column
from tensorflow.keras import layers

feature_columns = []
# numeric cols
for header in ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]:
    feature_columns.append(feature_column.numeric_column(header))

In [118]:
column_names = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"]

In [121]:
def preprocess_csv_line(*line):
    features = dict(zip(column_names[:-1], line[:-1]))
    label = line[-1]
    return features, label

In [192]:
batch_size = 32
num_epochs = 10

dataset = tf.data.experimental.make_csv_dataset(
    "heart.csv",
    batch_size=batch_size,
    column_names=column_names,
    label_name=column_names[-1],
    num_epochs=num_epochs,
    header=True,
    shuffle=True,
).map(preprocess_csv_line)

In [193]:
dataset

<MapDataset element_spec=({'age': OrderedDict([('age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('sex', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('cp', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('trestbps', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('chol', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('fbs', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('restecg', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thalach', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('exang', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('oldpeak', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('slope', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('ca', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thal', TensorSpec(shape=(None,), dtype=tf.int32, name=None))])}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [199]:
for elem in dataset:
    print(elem)

({'age': OrderedDict([('age', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([41, 65, 76, 70, 40, 55, 51, 69, 65, 58, 53, 44, 51, 60, 45, 44, 59,
       63, 59, 57, 63, 57, 43, 68, 47, 59, 57, 53, 71, 52, 58, 60])>), ('sex', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1])>), ('cp', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 0, 2, 0, 0, 0, 0, 3, 2, 2, 0, 1, 2, 3, 0, 2, 0, 0, 0, 1, 3, 2,
       0, 0, 2, 2, 0, 0, 0, 2, 3, 0])>), ('trestbps', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([135, 110, 140, 145, 152, 132, 140, 160, 155, 132, 130, 130, 100,
       150, 142, 108, 174, 130, 110, 154, 145, 150, 115, 144, 138, 126,
       150, 123, 112, 172, 150, 125])>), ('chol', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([203, 248, 197, 174, 223, 353, 261, 234, 269, 224, 264, 219, 222,
       240, 309, 141, 249, 254, 239, 232, 233, 126, 303, 193, 257, 218,
     

In [183]:
for key, value in dataset:
    print(key)

{'age': OrderedDict([('age', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([48, 58, 63, 71, 57, 61, 38, 48, 56, 59, 45, 62, 41, 44, 40, 71, 52,
       76, 63, 69, 47, 58, 63, 35, 68, 47, 46, 41, 56, 59, 51, 46])>), ('sex', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0])>), ('cp', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 2, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 1, 2, 2, 1, 1, 1, 3, 0, 2])>), ('trestbps', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([110, 105, 135, 160, 130, 148, 138, 124, 130, 138, 138, 150, 126,
       130, 152, 110, 108, 140, 150, 140, 112, 120, 140, 122, 180, 130,
       101, 110, 120, 178, 140, 142])>), ('chol', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([229, 240, 252, 302, 131, 203, 175, 274, 283, 271, 236, 244, 306,
       233, 223, 265, 233, 197, 407, 254, 204, 340, 187, 192, 274, 253,
      

In [184]:
for key, value in dataset:
    print(value)

tf.Tensor([1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 

In [142]:
model = tf.keras.Sequential([
  layers.Dense(128, activation='relu', input_shape=(13,)),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [143]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 128)               1792      
                                                                 
 dense_39 (Dense)            (None, 128)               16512     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_40 (Dense)            (None, 1)                 129       
                                                                 
Total params: 18,433
Trainable params: 18,433
Non-trainable params: 0
_________________________________________________________________


In [186]:
model.fit(dataset, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\input_spec.py", line 197, in assert_input_compatibility
        raise V